In [1]:
from name_parsing import NameAddressParser
# parser = NameAddressParser("../models/onnx/quantized")
parser = NameAddressParser("../models/onnx/onnx_export")
result = parser.parse(
    "Robert Chen 450 Maplewood Dr, Springfield IL 62704"
)
result

/Users/alexchen/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'first_name': 'Robert', 'last_name': 'Chen', 'street_name': 'Maplewood'}

In [ ]:

import pandas as pd
pd.set_option('display.max_colwidth', None)
strings = [
    'Robert Chen 450 Maplewood Dr, Springfield IL 62704',
    'JohnDoe 450 Maplewood Dr, Springfield IL 62704',
    'sarah martinez, 37/harbor way, coastal city, ca, 90210',
    'kim y. wong david r. wong 7197 260th Street new hyde park, ny 11040-1423',
    'james t. parker lisa r. parker 90 silver lake dr. port richmond sta., ny 10301',
    'mike reynolds dba reynolds consulting 2100 clearwater blvd tampa, fl 33601',
    "JJ's Sushi LLC, 90 silver lake dr. port richmond sta., ny 10301"
]

output = [parser.parse(i) for i in strings]
pd.DataFrame({'str': strings, 'output': output})

,str,output
0,"Robert Chen 450 Maplewood Dr, Springfield IL 62704","{'first_name': 'Robert', 'last_name': 'Chen', 'street_name': 'Maplewood'}"
1,"JohnDoe 450 Maplewood Dr, Springfield IL 62704","{'first_name': 'JohnDoe', 'last_name': '', 'street_name': 'Maplewood'}"
2,"sarah martinez, 37/harbor way, coastal city, ca, 90210","{'first_name': 'sarah', 'last_name': 'martinez', 'street_name': ''}"
3,"kim y. wong david r. wong 7607 270th Street new hyde park, ny 11040-1417","{'first_name': 'kim', 'last_name': '', 'street_name': '270th'}"
4,"james t. parker lisa r. parker 90 silver lake dr. port richmond sta., ny 10301","{'first_name': 'james', 'last_name': 'parker', 'street_name': ''}"
5,"mike reynolds dba reynolds consulting 2100 clearwater blvd tampa, fl 33601","{'first_name': 'mike', 'last_name': '', 'street_name': ''}"
6,"JJ's Sushi LLC, 90 silver lake dr. port richmond sta., ny 10301","{'first_name': 'JJ', 'last_name': 'LLC', 'street_name': 'silver'}"


In [6]:
from name_parsing.config import ID2LABEL
import numpy as np
text = "Robert Chen 450 Maplewood Dr, Springfield IL 62704"
words = text.split()
encoding = parser.tokenizer(
    words, is_split_into_words=True, return_tensors="np")
outputs = parser.session.run(
    None, {"input_ids": encoding["input_ids"], "attention_mask": encoding["attention_mask"]})
preds = np.argmax(outputs[0][0], axis=-1).tolist()
word_ids = encoding.word_ids()

seen = set()
for pred, wid in zip(preds, word_ids):
    if wid is not None and wid not in seen:
        seen.add(wid)
        print(f"  {words[wid]!r:20s} → {ID2LABEL[pred]}")

  'Robert'             → B-FIRST_NAME
  'Chen'               → O
  '450'                → O
  'Maplewood'          → B-STREET_NAME
  'Dr,'                → O
  'Springfield'        → O
  'IL'                 → O
  '62704'              → O
